In [ ]:
#!pip install pandas numpy emoji langdetect transformers torch


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import csv
import os
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.cluster import KMeans
import numpy as np
import torch
import torch.nn.functional as F
import urllib.request
from scipy.special import softmax

c:\Users\HRMic\OneDrive - ADvTECH Ltd\Documents\VS code\Python_VS\TWITTER\Research\gpuEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
df = pd.read_csv('2020_Classified_Balanced.csv')
df.head()

,label,tweet
0,Left,@ USER have the big rating in the US @ USER be...
1,Right,trump even try to taint Christmas with his cor...
2,Right,@ USER @ USER Republicans have to admit @ USER...
3,Right,We believe in a government with limited power ...
4,Right,Presidential Debate Round 2 Clear debate winne...


## **Roberta Sentiment MODEL**
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [13]:
# load model and tokenizer
task = 'sentiment' # set the task to sentiment
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"# specify the model name based on the task
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, use_safetensors=True).to('cuda')


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# load label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
print("Labels:", labels)
# (Barbieri et al., 2020; cardiffnlp, 2020)

Labels: ['negative', 'neutral', 'positive']


In [16]:
# preprocess text (username and link replacement)
# to follow consistency with the model's training data
def preprocess(text): # (Barbieri et al., 2020; cardiffnlp, 2020)
    new_text = []
    for t in str(text).lower().split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [17]:
# function to get sentiment
def getSentiment(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128).to('cuda')
    with torch.no_grad():
        output = model(**encoded_input)
        scores = output.logits[0].detach().cpu().numpy()
        scores = softmax(scores)
    pred_id = np.argmax(scores)
    # return label and scores
    return labels[pred_id], scores
# (Barbieri et al., 2020; cardiffnlp, 2020)

In [19]:
# apply the function to the DataFrame
sentOutput = df["tweet"].apply(getSentiment)
# split tuple results into separate columns
df["sentiment_label"] = sentOutput.apply(lambda x: x[0])
df["sentiment_probs"] = sentOutput.apply(lambda x: x[1])
df.head()
# (Barbieri et al., 2020; cardiffnlp, 2020)

,label,tweet,sentiment_label,sentiment_probs
0,Left,@ USER have the big rating in the US @ USER be...,neutral,"[0.3348276, 0.5837176, 0.08145483]"
1,Right,trump even try to taint Christmas with his cor...,negative,"[0.9129567, 0.08335175, 0.003691536]"
2,Right,@ USER @ USER Republicans have to admit @ USER...,negative,"[0.8504399, 0.13069777, 0.018862283]"
3,Right,We believe in a government with limited power ...,neutral,"[0.0368806, 0.60680854, 0.35631084]"
4,Right,Presidential Debate Round 2 Clear debate winne...,neutral,"[0.28923002, 0.61598545, 0.09478454]"


In [20]:
labels = ['negative', 'neutral', 'positive']
sentiment_counts = df.groupby(['label', 'sentiment_label']).size().unstack(fill_value=0)
# convert counts to proportions for each political leaning
sentiment_avg = sentiment_counts.div(sentiment_counts.sum(axis=1), axis=0)

In [ ]:
# Save results
df.to_csv("2020_sentiment_robertaBase.csv", index=False)

**Results**

In [22]:
# average sentiment probabilities by political leaning
print("\nAverage Sentiment Distribution by Political Leaning:\n")
sentiment_avg



Average Sentiment Distribution by Political Leaning:



sentiment_label,negative,neutral,positive
label,,,
Left,0.451704,0.469129,0.079167
Right,0.577118,0.387136,0.035746
